In [2]:
%load_ext autoreload

In [5]:
import os
from dotenv import load_dotenv
import sys
import os

# Add the parent directory of 'src' to the Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# Load environment variables from .env file


In [6]:
from google.auth import default
credentials, project = default()

In [7]:
%autoreload 2
# Now you can import from your local modules
from src.phrase import correct_grammar

# Test the import
correct_grammar("He love you")

'He loves you'

In [ ]:


# Usage examples
from src.audio_generation import  text_to_speech, play_audio

In [ ]:
audio_seg = text_to_speech("hello world", language_code="en-GB", voice_name="en-GB-Studio-C")
play_audio(audio_seg)

In [9]:
from src.dialogue_generation import anthropic_generate

#anthropic_generate("test") now enabled

'Hello! I\'m an AI assistant ready to help you with any questions or tasks you might have. Since you just wrote "test," I\'m not sure if you have a specific question or if you were simply checking to see if I\'m responsive. Is there anything in particular you\'d like assistance with? Feel free to ask me about a wide range of topics or for help with various tasks.'